In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('../../app'))

In [3]:
from core.operators import vid_vec_rep_clip
from core.models.media_factory import VideoFactory
from core.store.es_vec import ES
from core.config import *
from core.models.media import MediaType
from datetime import datetime

In [11]:
video_links = [
    "https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/en-speech.mp4",
    "https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/hi-speech.mp4",
    "https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/ta-speech.mp4"
]

file_names = [x.split('/')[-1] for x in video_links]


In [5]:
def get_vector_representation_of_video(video_links):
    
    result = []
    vid_vec_rep_clip.initialize({})
    for link in video_links:
        video = VideoFactory.make_from_url(link)
        vector = vid_vec_rep_clip.run(video)
        result.append(vector)

    return result

final_embeddings = get_vector_representation_of_video(video_links)
        
    

Installing packages for vid_vec_rep_clip


/usr/app/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/app/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


100% [..............................................................................] 48488 / 48488
Video downloaded
100% [..............................................................................] 47732 / 47732
Video downloaded
100% [..............................................................................] 51309 / 51309
Video downloaded


In [6]:
# temp  = VideoFactory.make_from_url(
#     r"https://raw.githubusercontent.com/tattle-made/feluda/main/src/core/operators/sample_data/sample-cat-video.mp4"
# )

In [8]:
# vid_vec_rep_clip.initialize({})
# t = vid_vec_rep_clip.run(temp)

Installing packages for vid_vec_rep_clip


/usr/app/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/app/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Code to initialize Elastic search instance

param_dict = {
    "host_name": "es",
    "text_index_name": "text",
    "image_index_name": "image",
    "video_index_name": "video",
    "audio_index_name": "audio",
}

param = QueueConfig(
    label="test",
    type="es",
    parameters=StoreESParameters(
        host_name=param_dict["host_name"],
        image_index_name=param_dict["image_index_name"],
        text_index_name=param_dict["text_index_name"],
        video_index_name=param_dict["video_index_name"],
        audio_index_name=param_dict["audio_index_name"],
    ),
)
es = ES(param)
es.connect()

In [14]:
def generate_document(post_id: str, representation: any):
    base_doc = {
        "e_kosh_id": "",
        "dataset": post_id,
        "metadata": None,
        "date_added": datetime.now().isoformat(),
    }

    def generator_doc():
        for vector in representation:
            base_doc["_index"] = "video"
            base_doc["vid_vec"] = vector["vid_vec"]
            base_doc["is_avg"] = vector["is_avg"]
            yield base_doc

    return generator_doc


for file_name,embedding in zip(file_names,final_embeddings):
    print(file_name,embedding)
    doc = generate_document(file_name,embedding)
    media_type = MediaType.VIDEO
    result = es.store(media_type, doc)
    print("result:", result)


en-speech.mp4 <generator object gendata at 0x757f15c93920>
----> 6 (0, [])
result: {'message': 'multiple media stored'}
hi-speech.mp4 <generator object gendata at 0x757f15c93060>
----> 6 (0, [])
result: {'message': 'multiple media stored'}
ta-speech.mp4 <generator object gendata at 0x757f15c93840>
----> 6 (0, [])
result: {'message': 'multiple media stored'}


In [15]:
!curl -X GET "http://es:9200/_cat/indices?v"

health status index uuid                   pri rep docs.count docs.deleted store.size pri.store.size dataset.size
yellow open   video B1F4p1UGTMy-lFPMjw8g3g   1   1         24            0    270.9kb        270.9kb      270.9kb
